In [1]:
import pandas as pd
import sklearn
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import root_mean_squared_error
import joblib

In [ ]:


# Charger le csv
df = pd.read_csv("../vessel-total-clean-final.csv")

# Convertir la colonne date en datetime
df['base_date_time'] = pd.to_datetime(df['base_date_time'])

# Trier par mmsi + date (si plusieurs navires)
df = df.sort_values(['mmsi', 'base_date_time'])

# Calculer delta_seconds pour chaque MMSI
df['delta_seconds'] = df.groupby('mmsi')['base_date_time'].diff().dt.total_seconds()

df = df.groupby('mmsi').apply(lambda group: group.iloc[1:]).reset_index(drop=True)

print(df.columns.tolist())





['id', 'mmsi', 'base_date_time', 'lat', 'lon', 'sog', 'cog', 'heading', 'vessel_name', 'imo', 'call_sign', 'vessel_type', 'status', 'length', 'width', 'draft', 'cargo', 'transceiver_class', 'delta_seconds']


/var/folders/58/f827j2cx7n12q31tb_gc35lw0000gn/T/ipykernel_66678/586801013.py:15: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  df = df.groupby('mmsi').apply(lambda group: group.iloc[1:]).reset_index(drop=True)


Creation des labels y

In [27]:
df['lat_next'] = df.groupby('mmsi')['lat'].shift(-1)
df['lon_next'] = df.groupby('mmsi')['lon'].shift(-1)

In [28]:
df = df.dropna(subset=['lat_next', 'lon_next'])

In [40]:
features = ['sog', 'cog', 'heading', 'length', 'draft', 'delta_seconds']
X = df[features]
y = df[['lat_next', 'lon_next']]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, shuffle=False)


model = RandomForestRegressor()
model.fit(X_train, y_train)

y_pred = model.predict(X_test)


rmse = root_mean_squared_error(y_test, y_pred)
print("RMSE:", rmse)

RMSE: 3.382615147923357


In [42]:
joblib.dump(model, "trajectoire.pkl")

['trajectoire.pkl']